In [1]:
import pandas as pd
import numpy as np
#%pip install scikit-learn


In [ ]:
df_mvp_tracks = pd.read_csv(r'C:\Users\Cometaca\Music_recommendation\new_tracks\df_mvp_tracks', index_col= [0])


Use the previous model and scaler for predicting a subset of the dataframe

In [ ]:
import joblib

audio_features = ['instrumentalness', 'liveness', 'speechiness', 'danceability',
                  'valence', 'loudness', 'tempo', 'acousticness', 'energy', 'mode', 'key']

model_rf = joblib.load(r'C:\Users\Cometaca\Music_recommendation\Models\model_rf.pkl')
scaler = joblib.load(r'C:\Users\Cometaca\Music_recommendation\Models\scaler.pkl')

df = df_mvp_tracks.sample(5000)
X_test = df[audio_features]
X_scaled= scaler.fit_transform(X_test)

df['predicted_sentiments'] = model_rf.predict(X_scaled)


In [39]:
from sklearn.metrics.pairwise import cosine_similarity

predicted_rows = df[df['predicted_sentiments']==1]


def recommend_songs(user_id, predicted_rows, audio_features, n):
    # Filter data for the chosen user
    user_tracks = df_mvp_tracks[df_mvp_tracks['user_id'] == user_id]
    #Get the average of user's aduio features
    user_avg_audio_features = user_tracks[audio_features].mean().values.reshape(1, -1)
    
    #Find cosine similarity between audio features of user's previous data and new predicted subset
    recommendations = []
    all = []
    i= 0
    for _, row in predicted_rows.iterrows():
        
        candidates = row[audio_features].values.reshape(1, -1)
        if row['track_id'] not in user_tracks['track_id']:
            similarity = cosine_similarity(candidates, user_avg_audio_features)
            recommendations = (row['track_name'], row['artists'], row['album_name'], similarity)
            if recommendations not in all:
                all.append(recommendations)
    all = sorted(all, key=lambda x: x[3], reverse=True)[:n]

    return all

# Choose a random user from the remaining dataset
df_rest = df_mvp_tracks[~df_mvp_tracks.index.isin(df.index)]
random_users = np.random.choice(df_rest['user_id'], size=5)

# Get recommendations for the chosen random user
for user in random_users:
    recommendations = recommend_songs(user, predicted_rows, audio_features, n=2)
    recommendations_df = pd.DataFrame(recommendations, columns=['Track', 'Artist', 'Album', 'Similarity Score'])
    print(f"Recommendations for User {user}:\n")
    print(recommendations_df)



Recommendations for User 23247402:

               Track              Artist                         Album  \
0    I'm Good (Blue)       Kidz Bop Kids                 KIDZ BOP 2023   
1  Five Little Ducks  Super Simple Songs  Baby Shark & More Kids Songs   

         Similarity Score  
0  [[0.9999895841731624]]  
1  [[0.9999868876090343]]  
Recommendations for User 863131741:

                 Track  Artist                Album        Similarity Score
0              abcdefu   GAYLE              abcdefu  [[0.9999950599272903]]
1  1553470665499594756  frxgxd  1553470665499594756  [[0.9999927334549278]]
Recommendations for User 15518784:

                    Track           Artist                   Album  \
0           Daddy Says No  Haschak Sisters           Daddy Says No   
1  414bigfrank (Backpack)      414bigfrank  414bigfrank (Backpack)   

         Similarity Score  
0  [[0.9999946615642286]]  
1  [[0.9999929018426512]]  
Recommendations for User 17945688:

           Track         